In [10]:
!pip install gradio

In [44]:
import os
import gradio as gr
import sqlite3
from dotenv import load_dotenv
import google.generativeai as genai

# Load API key and model
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("models/gemini-1.5-pro-latest")

# Setup SQLite DB
conn = sqlite3.connect("mistakes.db", check_same_thread=False)
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS mistakes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_input TEXT,
    correction TEXT
)
""")
conn.commit()

# Chat logic

def chat_with_bot(user_input, learning_lang, known_lang_list, level, history):
    known_lang = ", ".join(known_lang_list) if isinstance(known_lang_list, list) else known_lang_list
    prompt = f"""
You are a friendly language tutor.
The user is a {level} level learner learning {learning_lang} and knows {known_lang}.
Here is the user's sentence: "{user_input}"

Instructions:
- If incorrect:
    Mistake: Yes
    Correction: [...]
    Reply: [...]
- If correct:
    Mistake: No
    Reply: [...]
- ⚠️ Must follow format strictly.
"""
    try:
        response = model.generate_content(prompt).text.strip()
        if "Mistake: Yes" in response:
            correction = response.split("Correction:")[1].split("Reply:")[0].strip()
            reply = response.split("Reply:")[1].strip()
            cursor.execute("INSERT INTO mistakes (user_input, correction) VALUES (?, ?)", (user_input, correction))
            conn.commit()
        else:
            reply = response.split("Reply:")[-1].strip()
    except Exception as e:
        reply = f"Oops! Couldn't understand the tutor's reply.\n\nDEBUG: {e}"

    history.append((user_input, reply))
    return "", history

# View and clear

def view_mistakes():
    cursor.execute("SELECT * FROM mistakes")
    rows = cursor.fetchall()
    return "\n".join([f"ID: {r[0]}\nUser: {r[1]}\nCorrection: {r[2]}\n---" for r in rows]) if rows else "No mistakes logged yet."

def clear_mistakes():
    cursor.execute("DELETE FROM mistakes")
    conn.commit()
    return "All mistakes cleared."

# Language options for dropdowns
languages = sorted([
    "Arabic", "Bengali", "Chinese", "Dutch", "English", "French", "German", "Gujarati",
    "Hebrew", "Hindi", "Italian", "Japanese", "Kannada", "Korean", "Malayalam", "Marathi",
    "Portuguese", "Punjabi", "Russian", "Spanish", "Tamil", "Telugu", "Turkish", "Urdu", "Vietnamese"
])

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 🗣️ Language Learning Chatbot (Gemini)")
    gr.Markdown("Corrects your grammar gently while chatting in your learning language.")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your message")
    learning = gr.Dropdown(choices=languages, label="Language to learn", allow_custom_value=True, filterable=True)
    known = gr.CheckboxGroup(choices=languages, label="Languages you know")
    level = gr.Dropdown(["Beginner", "Intermediate", "Advanced"], label="Your level", value="Beginner")

    with gr.Row():
        send = gr.Button("Send")
        clear = gr.Button("Clear Chat")
    with gr.Row():
        view = gr.Button("View Mistakes")
        reset = gr.Button("Clear Mistakes")

    log = gr.Textbox(label="Mistake Log", lines=10)

    send.click(chat_with_bot, [msg, learning, known, level, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)
    view.click(view_mistakes, None, log)
    reset.click(clear_mistakes, None, log)

# Run app
demo.launch()

C:\Users\PC\AppData\Local\Temp\ipykernel_18828\4058074085.py:82: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
